In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
df = pd.read_csv('C:\\Users\\Huihao Xing\\Desktop\\G1\\DS 701\\Project\\Dataset\\arrdep_18augsep_simplecleaned.csv')

In [45]:
df

,service_date,route_id,direction,half_trip_id,stop_id,time_point_id,time_point_order,point_type,standard_type,scheduled,actual,scheduled_headway,headway,scheduled_nextday,actual_nextday
0,2018-08-01,01,Inbound,40121394.0,75,mit,4.0,Midpoint,Schedule,05:19:00,05:19:34,NaN,NaN,False,False
1,2018-08-01,01,Inbound,40121394.0,79,hynes,5.0,Midpoint,Schedule,05:22:00,05:23:20,NaN,NaN,False,False
2,2018-08-01,01,Inbound,40121394.0,187,masta,6.0,Midpoint,Schedule,05:25:00,05:25:58,NaN,NaN,False,False
3,2018-08-01,01,Inbound,40121394.0,59,Wasma,7.0,Midpoint,Schedule,05:28:00,05:28:26,NaN,NaN,False,False
4,2018-08-01,01,Inbound,40121565.0,110,hhgat,1.0,Startpoint,Headway,05:30:00,05:29:57,1200.0,1218.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753795,2018-09-30,SL5,Outbound,40832424.0,15176,mawor,8.0,Midpoint,Headway,00:54:00,01:27:08,900.0,2609.0,True,True
4753796,2018-09-30,SL5,Outbound,40832424.0,55,Wasma,9.0,Midpoint,Headway,00:55:00,01:27:46,900.0,2611.0,True,True
4753797,2018-09-30,SL5,Outbound,40832424.0,60,Walen,10.0,Midpoint,Headway,00:56:00,01:28:07,900.0,2610.0,True,True
4753798,2018-09-30,SL5,Outbound,40832424.0,61,Melwa,11.0,Midpoint,Headway,00:57:00,01:29:13,900.0,2615.0,True,True


In [46]:
# mean waiting time
df['scheduled'] = pd.to_datetime(df['scheduled'], format='%H:%M:%S', errors='coerce')
df['actual'] = pd.to_datetime(df['actual'], format='%H:%M:%S', errors='coerce')

df['wait_time'] = (df['actual'] - df['scheduled']).dt.total_seconds()

df_filtered = df.dropna(subset=['wait_time'])

on_time = df_filtered[df_filtered['wait_time'] <= 0]['wait_time'].mean()  
delayed = df_filtered[df_filtered['wait_time'] > 0]['wait_time'].mean()  

print(f"Average waiting time for on-time buses: {abs(on_time)} seconds")
print(f"Average waiting time for delayed buses: {delayed} seconds")


Average waiting time for on-time buses: 745.0745501646979 seconds
Average waiting time for delayed buses: 382.796515848305 seconds


In [47]:
# mean delay - all routes

average_delay_all_routes = df_filtered[df_filtered['wait_time'] > 0]['wait_time'].mean()

average_delay_all_routes

382.796515848305

In [48]:
# mean delay - taget routes
target_routes = ['22', '29', '15', '45', '28', '44', '42', '17', '23', '31', '26', '111', '24', '33', '14']

target_route_data = df_filtered[df_filtered['route_id'].isin(target_routes)]

average_delay_target_routes = target_route_data[target_route_data['wait_time'] > 0]['wait_time'].mean()

average_delay_target_routes

380.1467809037115

In [49]:
# disparity in service level
route_delays = df_filtered.groupby('route_id').agg(
    total_buses=('wait_time', 'count'),                       # Total number of buses per route
    delayed_buses=('wait_time', lambda x: (x > 0).sum())      # Number of delayed buses per route
).reset_index()

route_delays['delay_percentage'] = (route_delays['delayed_buses'] / route_delays['total_buses']) * 100

route_delays_sorted = route_delays.sort_values(by='delay_percentage', ascending=False)

print(route_delays_sorted.head(15)) 
print('\n')
print(route_delays_sorted.tail(15)) 

    route_id  total_buses  delayed_buses  delay_percentage
49       215        29712          28663         96.469440
59       236        13216          12466         94.325061
18       114         2740           2573         93.905109
54       221         1638           1538         93.894994
10       105         8530           8000         93.786635
63       245        10233           9570         93.520962
160       91        16041          14968         93.310891
87       411        12294          11444         93.086058
109      459        12472          11562         92.703656
15       110        35562          32899         92.511670
166       97         8282           7655         92.429365
65        27        10930          10100         92.406221
43       202        10734           9891         92.146451
169      CT2        27065          24929         92.107888
47       212         3770           3468         91.989390


    route_id  total_buses  delayed_buses  delay_percen